In [4]:
import cv2
import numpy as np
import mediapipe as mp
import screen_brightness_control as sbc
from math import hypot
import pyautogui
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time

In [22]:
# Accuracy tracking variables
last_brightness = 0
last_volume = 0

def main():
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    volRange = volume.GetVolumeRange()
    minVol, maxVol, _ = volRange

    mpHands = mp.solutions.hands
    hands = mpHands.Hands(
        static_image_mode=False,
        model_complexity=1,
        min_detection_confidence=0.75,
        min_tracking_confidence=0.75,
        max_num_hands=2)

    draw = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)

    last_brightness = 0
    last_volume = 0
    zoom_distance_last = 0

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)

            left_landmark_list, right_landmark_list = get_left_right_landmarks(frame, processed, draw, mpHands)

            brightness_accuracy = "Stable"
            volume_accuracy = "Stable"
            zoom_accuracy = "Stable"

            # Change brightness using left hand (In video it would appear as right hand as we are mirroring the frame)
            if left_landmark_list:
                left_distance = get_distance(frame, left_landmark_list)
                b_level = np.interp(left_distance, [50, 220], [0, 100])
                sbc.set_brightness(int(b_level))

                # Check accuracy
                if abs(b_level - last_brightness) > 5:  # Adjust threshold as needed
                    brightness_accuracy = f"{int(b_level)}%"
                    last_brightness = b_level

            # Change volume using right hand (In video it would appear as left hand as we are mirroring the frame)
            if right_landmark_list:
                right_distance = get_distance(frame, right_landmark_list)
                vol = np.interp(right_distance, [50, 220], [minVol, maxVol])
                volume.SetMasterVolumeLevel(vol, None)

                # Check accuracy
                volume_percentage = int(np.interp(vol, [minVol, maxVol], [0, 100]))
                if abs(volume_percentage - last_volume) > 5:  # Adjust threshold as needed
                    volume_accuracy = f"{volume_percentage}%"
                    last_volume = volume_percentage

                # Zoom control using thumb and middle finger
                right_zoom_distance = get_distance(frame, right_landmark_list, index1=4, index2=12)  # Thumb and Middle finger
                zoom_accuracy = zoom_control(right_zoom_distance)  # Trigger zoom based on distance
                zoom_distance_last = right_zoom_distance  # Save the last zoom distance

            # Print accuracy statuses
            print(f"Brightness Accuracy: {brightness_accuracy}")
            print(f"Volume Accuracy: {volume_accuracy}")
            print(f"Zooming Accuracy: {zoom_accuracy}")

            cv2.imshow('Image', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

def check_brightness_accuracy(current_brightness):
    global last_brightness
    if abs(current_brightness - last_brightness) > 10:  # Threshold for significant change
        print(f"Brightness changed to {current_brightness} (accurate)")
        last_brightness = current_brightness
    else:
        print("Brightness adjustment inaccurate or redundant")

def check_volume_accuracy(current_volume, minVol, maxVol):
    global last_volume
    volume_percentage = np.interp(current_volume, [minVol, maxVol], [0, 100])
    if abs(volume_percentage - last_volume) > 5:  # Threshold for significant change
        print(f"Volume changed to {int(volume_percentage)}% (accurate)")
        last_volume = volume_percentage
    else:
        print("Volume adjustment inaccurate or redundant")
def zoom_control(distance):
    """Simulate browser zoom in/out using pyautogui and return zoom accuracy."""
    # Define distance thresholds
    zoom_in_threshold = 70  # Adjust these values based on your hand size and distance
    zoom_out_threshold = 200
    zoom_accuracy = "Stable"

    if distance < zoom_in_threshold:
        # Simulate Ctrl + '+' for zooming in
        pyautogui.hotkey('ctrl', '+')
        zoom_accuracy = f"Zooming In (Distance: {distance})"
        print("Zooming In")
    elif distance > zoom_out_threshold:
        # Simulate Ctrl + '-' for zooming out
        pyautogui.hotkey('ctrl', '-')
        zoom_accuracy = f"Zooming Out (Distance: {distance})"
        print("Zooming Out")
    else:
        zoom_accuracy = "Stable"

    return zoom_accuracy


In [23]:
if __name__=='__main__':
    main();

Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable
Brightness Accuracy: Stable
Volume Accuracy: Stable
Zooming Accuracy: Stable